In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

### Lendo o banco de dados

In [4]:
dados = pd.read_csv('train.csv')
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
dados  = dados.drop(['Name','Ticket','Cabin','Embarked'], axis=1)
dados.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


### Editando Chave e Variável Resposta

In [6]:
dados = dados.set_index(['PassengerId'])
dados = dados.rename(columns = {'Survived':'target'}, inplace=False)
dados.head() 

,target,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,male,22.0,1,0,7.2500
2,1,1,female,38.0,1,0,71.2833
3,1,3,female,26.0,0,0,7.9250
4,1,1,female,35.0,1,0,53.1000
5,0,3,male,35.0,0,0,8.0500


### Descritiva

In [7]:
dados.describe()

,target,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
# dados.describe(include=['0'])

### Tranformação dos dados

In [14]:
dados['Sex_F'] = np.where(dados['Sex'] == 'female',1,0)

dados['Pclass_1'] = np.where(dados['Pclass'] == 1, 1, 0)
dados['Pclass_2'] = np.where(dados['Pclass'] == 2, 1, 0)
dados['Pclass_3'] = np.where(dados['Pclass'] == 3, 1, 0)

In [15]:
dados = dados.drop(['Pclass','Sex'], axis=1)
dados.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1
2,1,38.0,1,0,71.2833,1,1,0,0
3,1,26.0,0,0,7.9250,1,0,0,1
4,1,35.0,1,0,53.1000,1,1,0,0
5,0,35.0,0,0,8.0500,0,0,0,1


In [16]:
dados.isnull().sum()

target        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex_F         0
Pclass_1      0
Pclass_2      0
Pclass_3      0
dtype: int64

In [17]:
dados.fillna(0, inplace=True)

In [18]:
dados.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

### Amostragem

In [20]:
x_train, x_test, y_train, y_test = train_test_split(dados.drop(['target'], axis=1),
                                                    dados['target'],
                                                    test_size=0.3,
                                                    random_state=1234)
[{'treino':x_train.shape}, {'teste':x_test.shape}]

[{'treino': (623, 8)}, {'teste': (268, 8)}]

### Modelo

In [21]:
rndforest = RandomForestClassifier(n_estimators=1000,
                                   criterion='gini',
                                   max_depth=5)
rndforest.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000)

In [22]:
probabilidade = rndforest.predict_proba(dados.drop('target', axis=1))[:,1]
classificacao = rndforest.predict(dados.drop('target', axis=1))

In [24]:
dados['probabilidade'] = probabilidade
dados['classificacao'] = classificacao
dados

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,probabilidade,classificacao
PassengerId,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1,0.123948,0
2,1,38.0,1,0,71.2833,1,1,0,0,0.935078,1
3,1,26.0,0,0,7.9250,1,0,0,1,0.449442,0
4,1,35.0,1,0,53.1000,1,1,0,0,0.920862,1
5,0,35.0,0,0,8.0500,0,0,0,1,0.142800,0
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.0,0,0,13.0000,0,0,1,0,0.128518,0
888,1,19.0,0,0,30.0000,1,1,0,0,0.847548,1
889,0,0.0,1,2,23.4500,1,0,0,1,0.481057,0
